In [ ]:
from scipy.integrate import ode
from scipy.integrate import odeint
import scipy
import numpy as np
import matplotlib.pyplot as plt
import control
import copy

This notebook is supposed to work out the differences between the continuous, exact solution and the discretization. Both solutions are plotted for a constant and a sinusoidal input signal. For the system the Python Conrol package is used.

In [ ]:
# define time vector
ts = 1e-4
t_end = 0.1 # <---- Change this, to get a longer timehorizon.
# For the transient behavior at the beginning a length of 0.005 is suitable.
steps = int(1/ts)
t = np.arange(0, t_end+ts, ts)
num_samples = len(t)

f0 = 50
V_eff = 230 * np.sqrt(2)

v_sin1 = V_eff * np.sin(2*np.pi * f0 * t)
v_sin2 = V_eff * np.sin(2*np.pi * f0 * t + 1)

vi1 = 225
vi2 = 235

R = 0.4
L = 2.3e-3
C = 10e-6
LT = 2.3e-3
RLoad = 14

#def varaibles

R1 = R
L1 = L
C1 = C
LT1 = LT
R2 = R
L2 = L
C2 = C
LT2 = LT
RT1 = R
RT2 = R

i10 = 0
v10 = 0
iT10 = 0
i20 = 0
v20 = 0
iT20 = 0
t0 = 0

x0 = np.array([i10, v10, iT10, i20, v20, iT20])

In [ ]:
# create the input signals and plot them
u = np.array([vi1, vi2])[:,None] * np.ones((2,len(t)))
u_sin = np.array([v_sin1, v_sin2])

plt.plot(t, v_sin1)
plt.plot(t, v_sin2)
plt.plot(t, u[0])
plt.plot(t, u[1], '--')
plt.ylabel('Input current in $v$')
plt.xlabel('Time in $s$')
plt.grid()
plt.show()

## Calculating the continous case 

In [ ]:
A1 = np.array([[-R1/L1, -1/L1, 0],
               [1/C1, 0, -1/C1],
               [0, 1/LT1, -RLoad/LT1 - RT1/LT1]])

A2 = np.array([[-R2/L2, -1/L2, 0],
               [1/C2, 0, -1/C2],
               [0, 1/LT2, -RLoad/LT2 - RT2/LT2]])

AT1 = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, -RLoad/LT1]])

AT2 = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, -RLoad/LT2]])

A = np.block([[A1, AT1],
              [AT2, A2]])

In [ ]:
zeros = np.zeros((3,1))
B1 = np.array([[1/L1],
              [0],
              [0]])

B2 = np.array([[1/L2],
              [0],
              [0]])

B = np.block([[B1, zeros],
             [zeros, B2]])

In [ ]:
C1 = np.array([[1, 1, 1]])
C2 = np.array([[1, 1, 1]])
C = np.block([C1, C2])

In [ ]:
sys = control.ss(A, B, C, 0)

### Constant input

In [ ]:
T, yout, xout_c = control.forced_response(sys, T=t, U=u, X0=x0, return_x=True, squeeze=True)

### Sinus input

In [ ]:
T, yout, xout_c_sin = control.forced_response(sys, T=t, U=u_sin, X0=x0, return_x=True, squeeze=True)

## Calculating the discret case 

In [ ]:
sys_d = sys.sample(ts)

In [ ]:
A_d = scipy.linalg.expm(A*ts)

In [ ]:
sys_d.A == A_d # Check if discretization worked propertly

### Constant input

In [ ]:
T, yout, xout_d = control.forced_response(sys_d, T=t, U=u, X0=x0, return_x=True, squeeze=True)

### Sinus input

In [ ]:
T, yout, xout_d_sin = control.forced_response(sys_d, T=t, U=u_sin, X0=x0, return_x=True, squeeze=True)

# Comparison

In [ ]:
xout_c[1] == xout_d[1], np.isclose(xout_c[1], xout_d[1])

In [ ]:
xout_c_sin[1] == xout_d_sin[1], np.isclose(xout_c_sin[1], xout_d_sin[1])

In [ ]:
# Calculating Error

In [ ]:
diff_const = np.abs(xout_c[1] - xout_d[1])

In [ ]:
MSE_const = 1/num_samples * np.sum(diff_const**2) 

In [ ]:
diff_sin = xout_c_sin[1] - xout_d_sin[1]
diff_sin_abs = np.abs(xout_c_sin[1] - xout_d_sin[1])

In [ ]:
MSE_sin = 1/num_samples * np.sum(diff_sin**2) 

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(15,10), sharex=True)
ax[0].plot(t, u[0], label='u1')
ax[0].plot(t, u[1], '--', label='u2')
ax[0].plot(t, xout_c[1], label='vi1 - continous')
ax[0].plot(t, xout_d[1],'--', label='vi1 - discret')
ax[0].set_ylabel('Currents ($V$)')
ax[0].grid(True)

ax[1].plot(t, diff_const, label='Dif')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Difference')
ax[1].set_ylim(0,1)
plt.legend()
plt.grid()
plt.show()

No diffrence between the continous and discret version of the system by constant inputsignal.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15,10), sharex=True)
ax[0].plot(t, v_sin1, label='u1')
ax[0].plot(t, v_sin2, '--', label='u2')
ax[0].plot(t, xout_c_sin[1], label='vi1 - continous')
ax[0].plot(t, xout_d_sin[1],'--', label='vi1 - discret')
ax[0].set_ylabel('Currents ($V$)')
ax[0].grid(True)

ax[1].plot(t, diff_sin, label='Dif')
ax[1].plot(t, diff_sin_abs, '--', label='Dif_abs')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Difference')
ax[1].set_ylim(-10,10)
plt.legend()
plt.grid()
plt.show()

A strong deviation can be observed at the start time, which then decays into a continuous cosine wave. The diffrence between the continuous and the discrete system can be attributed to the discretization. 

In [ ]:
MSE_const, MSE_sin